In [1]:
import os
import pandas as pd
import glob

In [2]:
#import files 
positions=pd.read_csv('/global/home/users/chandlersutherland/e14/data/all_NLR.bed', sep='\t', header=0, names=['Chromosome', 'start', 'end', 'gene', 'strand'], index_col=False)
positions

input_path='/global/scratch/users/chandlersutherland/e14/STAR_output/htseq_count'

htseq_counts = glob.glob(os.path.join(input_path, "*.tsv"))
htseq_counts

dfs = []
stats = []

for f in htseq_counts: 
    #load in dataframe, get the filename 
    df= pd.read_csv(f, sep = '\t', names=['Gene', 'count'], index_col=False)
    print('Location:', f)
    filename=f.split('/')[-1].replace('.tsv', '')
    print('File Name:', filename)
    
    #pull out the stats and save in a separate list of dfs 
    stat=df.loc[df['Gene'].str.startswith('_', na=False)==True]
    stat['filename'] = filename
    stats.append(stat)
    
    #remove stats, and add the gene counts together 
    df = df[df["Gene"].str.contains("_")==False]
    df['filename'] = filename
    dfs.append(df)



Location: /global/scratch/users/chandlersutherland/e14/STAR_output/htseq_count/SRR17281235.tsv
File Name: SRR17281235
Location: /global/scratch/users/chandlersutherland/e14/STAR_output/htseq_count/SRR17281236.tsv
File Name: SRR17281236
Location: /global/scratch/users/chandlersutherland/e14/STAR_output/htseq_count/SRR17281235_trimmed.tsv
File Name: SRR17281235_trimmed


/global/software/sl-7.x86_64/modules/langs/python/3.7/lib/python3.7/site-packages/ipykernel_launcher.py:22: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


Location: /global/scratch/users/chandlersutherland/e14/STAR_output/htseq_count/SRR17281234_trimmed.tsv
File Name: SRR17281234_trimmed
Location: /global/scratch/users/chandlersutherland/e14/STAR_output/htseq_count/SRR17281234.tsv
File Name: SRR17281234
Location: /global/scratch/users/chandlersutherland/e14/STAR_output/htseq_count/SRR17281233_trimmed.tsv
File Name: SRR17281233_trimmed
Location: /global/scratch/users/chandlersutherland/e14/STAR_output/htseq_count/SRR17281236_trimmed.tsv
File Name: SRR17281236_trimmed
Location: /global/scratch/users/chandlersutherland/e14/STAR_output/htseq_count/SRR17281233.tsv
File Name: SRR17281233
Location: /global/scratch/users/chandlersutherland/e14/STAR_output/htseq_count/SRR3465232.tsv
File Name: SRR3465232


In [3]:
hv_status = pd.DataFrame(columns=['Gene', 'HV'])
hv_status[['Gene', 'HV']] = positions.gene.str.split('_', expand = True)
hv_status


,Gene,HV
0,AT1G10920,0
1,AT1G12210,0
2,AT1G12220,0
3,AT1G12280,0
4,AT1G12290,0
...,...,...
127,AT5G51630,0
128,AT5G58120,0
129,AT5G63020,0
130,AT5G66900,0


In [44]:
def zero_my_hero(depth, hv_status):
    zeros = pd.merge(depth, hv_status)
  #return counts
    return zeros.groupby(['filename', 'HV']).agg(
        lambda x:x.eq(0).sum(),
  )

In [48]:
zero_sum = [zero_my_hero(i, hv_status) for i in dfs]
pd.concat(zero_sum).sort_values(['filename', 'HV'])

Gene  count
filename            HV             
SRR17281233         0      0     18
                    1      0      4
SRR17281233_trimmed 0      0      9
                    1      0      4
SRR17281234         0      0     17
                    1      0      2
SRR17281234_trimmed 0      0      8
                    1      0      2
SRR17281235         0      0     19
                    1      0      3
SRR17281235_trimmed 0      0      8
                    1      0      4
SRR17281236         0      0     12
                    1      0      3
SRR17281236_trimmed 0      0      8
                    1      0      3
SRR3465232          0      0     59
                    1      0     23

In [49]:
def merger2(depth, hv_status):
  counts = pd.merge(depth, hv_status)
  #return counts
  return counts.groupby(['filename', 'HV']).agg(
        mean_count=('count', 'mean'),
  )

In [50]:
summary2 = [merger2(i, hv_status) for i in dfs]
raw_counts = pd.concat(summary2)
raw_counts.sort_values('filename')

mean_count
filename            HV             
SRR17281233         1    796.200000
                    0    234.539216
SRR17281233_trimmed 1    774.633333
                    0    230.901961
SRR17281234         0    309.088235
                    1   1034.933333
SRR17281234_trimmed 0    305.245098
                    1   1006.800000
SRR17281235         0    175.617647
                    1    587.066667
SRR17281235_trimmed 1    570.833333
                    0    173.323529
SRR17281236         1   1188.933333
                    0    380.156863
SRR17281236_trimmed 0    375.637255
                    1   1156.833333
SRR3465232          0     50.441176
                    1      4.266667

In [51]:
all_stats = pd.concat(stats)
all_stats['Gene'] = all_stats.Gene.str.replace('_', ' ')

pd.pivot_table(all_stats, index='filename', columns='Gene')

count                           \
Gene                  alignment not unique   ambiguous   no feature   
filename                                                              
SRR17281233                         527923          31     14026577   
SRR17281233_trimmed                 572550          28     13825256   
SRR17281234                         508695          32     13384498   
SRR17281234_trimmed                 556526          30     13207416   
SRR17281235                         359446          19      9699902   
SRR17281235_trimmed                 402388          22      9571862   
SRR17281236                         479409          44     13714677   
SRR17281236_trimmed                 522843          39     13524875   
SRR3465232                         1057633           0     70496422   

                                                   
Gene                  not aligned   too low aQual  
filename                                           
SRR17281233                     0               0  
SRR17281233_trimmed             0               0  
SRR17281234                     0               0  
SRR17281234_trimmed             0               0  
SRR17281235                     0               0  
SRR17281235_trimmed             0               0  
SRR17281236                     0               0  
SRR17281236_trimmed             0               0  
SRR3465232                      0               0